In [1]:
import pandas as pd
import numpy as np 
from autorank import autorank, create_report, plot_stats
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
import sklearn.tree as tree
import os
from sklearn.metrics import accuracy_score
from scipy.stats import shapiro, wilcoxon
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures
import statsmodels.api as sm 
#euclidian distance
from scipy.spatial import distance
#warnings
import warnings
warnings.filterwarnings("ignore")
#import datasetmanager
from DatasetManager import DatasetManager
from sklearn.linear_model import LinearRegression

#read data and drop irrelevant columns
df = pd.read_csv('results.csv', index_col=0)
#LOD
df['LOD']=1

In [32]:
MLDLclassifiers = ['XGB','RF','LSTM','CNN']

##### later

In [33]:
for i in range(0,len(df2)):
    if df2.iloc[i].classifier in MLDLclassifiers:
        event = 0
        case = 0
        control=0
        
        event = int(eval(df2.iloc[i]['feature importance (event, case, control)'])[0])
        case = int(eval(df2.iloc[i]['feature importance (event, case, control)'])[1])
        control = int(eval(df2.iloc[i]['feature importance (event, case, control)'])[2])

        shapevent =int(eval(df2.iloc[i]['shap (event, case, control)'])[0])
        shapcase =int(eval(df2.iloc[i]['shap (event, case, control)'])[1])
        shapcontrol =int(eval(df2.iloc[i]['shap (event, case, control)'])[2])

        df2['feature importance (event, case, control)'].iloc[i] = [event,case,control]
        df2['shap (event, case, control)'].iloc[i] = [shapevent,shapcase,shapcontrol]
        if event+case+control != 10:
            df2.iloc[i]['feature importance (event, case, control)'] = [event+1,case,control]
#reform feature and shap column
for i in range(0,len(df2)):
    if df2.iloc[i].classifier in MLDLclassifiers:
        coef = distance.euclidean(df2['shap (event, case, control)'].iloc[i], df2['feature importance (event, case, control)'].iloc[i])
        df2['LOD'].iloc[i] = coef
    else:
        df2['LOD'].iloc[i]= 0.0

In [34]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [37]:
df2

,dataset_name,cls_encoding,classifier,AUC,#event columns,#case columns,#control columns,total cols,pars. event,pars. case,pars. control,total pars cols,perc_pars_event,perc_pars_case,perc_pars_control,perc_event_total,perc_case_total,perc_control_total,perc_pars_total,FC event,FC_case,FC control,monotonicity,"shap (event, case, control)","feature importance (event, case, control)",LOD
6,bpic2011_f1,agg,XGB,0.955704,136,730,175,1041,80.0,67.0,68.0,215.0,58.823529,9.178082,38.857143,37.209302,31.162791,31.627907,20.653218,34.521990,16.811004,50.433011,0.776626,"[4, 3, 3]","[1, 7, 2]",5.099020
7,bpic2011_f1,index,XGB,0.923112,2248,730,2368,5346,318.0,56.0,100.0,474.0,14.145907,7.671233,4.222973,67.088608,11.814346,21.097046,8.866442,42.384106,22.754288,43.301070,0.681377,"[3, 6, 1]","[1, 6, 3]",2.828427
18,bpic2011_f2,agg,XGB,0.979851,146,760,215,1121,79.0,65.0,73.0,217.0,54.109589,8.552632,33.953488,36.405530,29.953917,33.640553,19.357716,41.431412,13.929755,23.605036,0.745028,"[3, 5, 2]","[1, 7, 2]",2.828427
19,bpic2011_f2,index,XGB,0.902795,2660,760,3139,6559,308.0,79.0,110.0,497.0,11.578947,10.394737,3.504301,61.971831,15.895372,22.132797,7.577375,21.577203,19.456594,14.102054,0.675061,"[3, 6, 1]","[1, 7, 2]",2.449490
30,bpic2011_f3,agg,XGB,0.988525,132,670,166,968,72.0,39.0,48.0,159.0,54.545455,5.820896,28.915663,45.283019,24.528302,30.188679,16.425620,23.078496,9.771055,31.439084,0.815246,"[1, 6, 3]","[1, 7, 2]",1.414214
31,bpic2011_f3,index,XGB,0.976994,1871,670,1980,4521,269.0,43.0,88.0,400.0,14.377338,6.417910,4.444444,67.250000,10.750000,22.000000,8.847600,24.345871,21.954211,10.016353,0.723446,"[4, 6, 0]","[2, 8, 0]",2.828427
42,bpic2011_f4,agg,XGB,0.851161,143,759,204,1106,69.0,152.0,52.0,273.0,48.251748,20.026350,25.490196,25.274725,55.677656,19.047619,24.683544,54.251769,9.304499,7.195301,0.331712,"[2, 8, 0]","[1, 9, 0]",1.414214
43,bpic2011_f4,index,XGB,0.836163,2647,759,3064,6470,189.0,148.0,56.0,393.0,7.140159,19.499341,1.827676,48.091603,37.659033,14.249364,6.074189,44.052863,15.845681,8.436791,0.281409,"[3, 7, 0]","[2, 8, 0]",1.414214
54,bpic2015_1_f2,agg,XGB,0.913890,88,35,266,389,64.0,20.0,55.0,139.0,72.727273,57.142857,20.676692,46.043165,14.388489,39.568345,35.732648,16.160788,4.573421,92.124692,0.865405,"[3, 3, 4]","[2, 4, 4]",1.414214
55,bpic2015_1_f2,index,XGB,0.744475,1733,35,2448,4216,347.0,18.0,114.0,479.0,20.023081,51.428571,4.656863,72.442589,3.757829,23.799582,11.361480,41.858080,18.765381,31.911403,0.770478,"[6, 3, 1]","[2, 4, 4]",5.099020


## filter

In [35]:
underperforming = list(df.groupby('dataset_name')['AUC'].mean()[df.groupby('dataset_name')['AUC'].mean()<0.5].index)
print(underperforming)

['sepsis_cases_1']


In [36]:
# Filter out the low performing datasets (AUC<0.5)
#only the sepsis_cases_1 should be filtered out, as it only concerns the GLRM model for dataset BPIC2015_1
df = df[~df.dataset_name.isin(underperforming)]

In [78]:
#not representative for XAI purpose
df = df[~df.dataset_name.isin(list(set(df.groupby('dataset_name')['AUC'].max()[df.groupby('dataset_name')['AUC'].max()<0.8].index)))]

In [79]:
dataset_cols = ['classifier','cls_encoding','AUC','#event columns','#case columns','#control columns','pars. event','pars. case','pars. control','perc_pars_event','perc_pars_case','perc_pars_control','FC event','FC_case','FC control','monotonicity','LOD']

In [81]:
bpic2015_3 = ['bpic2015_3_f2']
bpic2015_3 = df[df.dataset_name.isin(bpic2015_3)].sort_values('AUC')
bpic2015_3 = bpic2015_3[dataset_cols]
bpic2015_3 = bpic2015_3.sort_values(by='AUC')

In [82]:
bpic2015_3

,classifier,cls_encoding,AUC,#event columns,#case columns,#control columns,pars. event,pars. case,pars. control,perc_pars_event,perc_pars_case,perc_pars_control,FC event,FC_case,FC control,monotonicity,LOD
77,GLRM,index,0.536088,1700,34,3083,1.00,0.0,2.0,0.058235,0.000000,0.064872,0.438203,82.989742,97.311025,1.000000,1
79,XGB,index,0.867993,1700,34,3083,376.00,21.0,146.0,22.117647,61.764706,4.735647,27.776118,6.742356,46.349965,0.783251,1
81,RF,index,0.895360,1700,34,3083,1317.00,33.0,1867.0,77.470588,97.058824,60.557898,12.757693,85.081167,85.081167,0.171616,1
73,LR,index,0.898250,1700,34,3083,823.00,33.0,1744.0,48.411765,97.058824,56.568278,18.802908,6.104970,5.676725,1.000000,1
75,LLM,index,0.905283,1700,34,3083,439.75,11.5,804.5,25.867647,33.823529,26.094713,12.887163,0.000000,98.048003,1.000000,1
76,GLRM,agg,0.937832,87,34,314,0.00,0.0,8.0,0.000000,0.000000,2.547771,0.000000,0.000000,89.522956,1.000000,1
82,LSTM,embeddings,0.938762,11,18,1,11.00,1.0,1.0,100.000000,5.555556,100.000000,3.738541,1.383280,3.723603,0.648606,1
83,CNN,embeddings,0.945554,11,18,1,11.00,1.0,1.0,100.000000,5.555556,100.000000,2.672449,1.039203,2.898515,-0.521182,1
78,XGB,agg,0.949260,87,34,314,62.00,21.0,74.0,71.264368,61.764706,23.566879,16.880789,3.077383,87.720347,0.874730,1
74,LLM,agg,0.956438,87,34,314,86.00,33.5,307.5,98.850575,98.529412,97.929936,1.254855,0.000000,92.739767,1.000000,1


In [83]:
bpic2011_2 = ['bpic2011_f2']
bpic2011_2 = df[df.dataset_name.isin(bpic2011_2)].sort_values('AUC')
bpic2011_2 = bpic2011_2[dataset_cols]
bpic2011_2 = bpic2011_2.sort_values(by='AUC')

In [84]:
bpic2011_2

,classifier,cls_encoding,AUC,#event columns,#case columns,#control columns,pars. event,pars. case,pars. control,perc_pars_event,perc_pars_case,perc_pars_control,FC event,FC_case,FC control,monotonicity,LOD
23,CNN,embeddings,0.771690,13,6,1,13.0,6.00,1.0,100.000000,100.000000,100.000000,2.856852,3.949377,3.949377,0.502905,1
22,LSTM,embeddings,0.866833,13,6,1,13.0,6.00,1.0,100.000000,100.000000,100.000000,1.511892,1.511892,1.511892,0.638673,1
17,GLRM,index,0.883467,2660,760,3139,2.0,4.00,2.0,0.075188,0.526316,0.063715,24.758118,68.045063,71.013917,1.000000,1
19,XGB,index,0.902795,2660,760,3139,308.0,79.00,110.0,11.578947,10.394737,3.504301,21.577203,19.456594,14.102054,0.675061,1
21,RF,index,0.938924,2660,760,3139,2061.0,760.00,1974.0,77.481203,100.000000,62.886270,15.904573,12.339298,46.666667,0.058018,1
15,LLM,index,0.941562,2660,760,3139,7.5,28.75,9.0,0.281955,3.782895,0.286716,40.781975,77.799867,19.642147,1.000000,1
13,LR,index,0.964114,2660,760,3139,39.0,148.00,57.0,1.466165,19.473684,1.815865,3.671306,0.662691,0.649437,1.000000,1
14,LLM,agg,0.972269,146,760,215,126.5,542.50,192.5,86.643836,71.381579,89.534884,44.864148,0.357853,9.065606,1.000000,1
16,GLRM,agg,0.975331,146,760,215,2.0,5.00,1.0,1.369863,0.657895,0.465116,31.888668,46.587144,40.742213,1.000000,1
18,XGB,agg,0.979851,146,760,215,79.0,65.00,73.0,54.109589,8.552632,33.953488,41.431412,13.929755,23.605036,0.745028,1


In [85]:
bpic2015_3

,classifier,cls_encoding,AUC,#event columns,#case columns,#control columns,pars. event,pars. case,pars. control,perc_pars_event,perc_pars_case,perc_pars_control,FC event,FC_case,FC control,monotonicity,LOD
77,GLRM,index,0.536088,1700,34,3083,1.00,0.0,2.0,0.058235,0.000000,0.064872,0.438203,82.989742,97.311025,1.000000,1
79,XGB,index,0.867993,1700,34,3083,376.00,21.0,146.0,22.117647,61.764706,4.735647,27.776118,6.742356,46.349965,0.783251,1
81,RF,index,0.895360,1700,34,3083,1317.00,33.0,1867.0,77.470588,97.058824,60.557898,12.757693,85.081167,85.081167,0.171616,1
73,LR,index,0.898250,1700,34,3083,823.00,33.0,1744.0,48.411765,97.058824,56.568278,18.802908,6.104970,5.676725,1.000000,1
75,LLM,index,0.905283,1700,34,3083,439.75,11.5,804.5,25.867647,33.823529,26.094713,12.887163,0.000000,98.048003,1.000000,1
76,GLRM,agg,0.937832,87,34,314,0.00,0.0,8.0,0.000000,0.000000,2.547771,0.000000,0.000000,89.522956,1.000000,1
82,LSTM,embeddings,0.938762,11,18,1,11.00,1.0,1.0,100.000000,5.555556,100.000000,3.738541,1.383280,3.723603,0.648606,1
83,CNN,embeddings,0.945554,11,18,1,11.00,1.0,1.0,100.000000,5.555556,100.000000,2.672449,1.039203,2.898515,-0.521182,1
78,XGB,agg,0.949260,87,34,314,62.00,21.0,74.0,71.264368,61.764706,23.566879,16.880789,3.077383,87.720347,0.874730,1
74,LLM,agg,0.956438,87,34,314,86.00,33.5,307.5,98.850575,98.529412,97.929936,1.254855,0.000000,92.739767,1.000000,1


In [86]:
frames = [bpic2015_3,bpic2011_2]

In [87]:
result = pd.concat(frames)

In [88]:
result

,classifier,cls_encoding,AUC,#event columns,#case columns,#control columns,pars. event,pars. case,pars. control,perc_pars_event,perc_pars_case,perc_pars_control,FC event,FC_case,FC control,monotonicity,LOD
77,GLRM,index,0.536088,1700,34,3083,1.00,0.00,2.0,0.058235,0.000000,0.064872,0.438203,82.989742,97.311025,1.000000,1
79,XGB,index,0.867993,1700,34,3083,376.00,21.00,146.0,22.117647,61.764706,4.735647,27.776118,6.742356,46.349965,0.783251,1
81,RF,index,0.895360,1700,34,3083,1317.00,33.00,1867.0,77.470588,97.058824,60.557898,12.757693,85.081167,85.081167,0.171616,1
73,LR,index,0.898250,1700,34,3083,823.00,33.00,1744.0,48.411765,97.058824,56.568278,18.802908,6.104970,5.676725,1.000000,1
75,LLM,index,0.905283,1700,34,3083,439.75,11.50,804.5,25.867647,33.823529,26.094713,12.887163,0.000000,98.048003,1.000000,1
76,GLRM,agg,0.937832,87,34,314,0.00,0.00,8.0,0.000000,0.000000,2.547771,0.000000,0.000000,89.522956,1.000000,1
82,LSTM,embeddings,0.938762,11,18,1,11.00,1.00,1.0,100.000000,5.555556,100.000000,3.738541,1.383280,3.723603,0.648606,1
83,CNN,embeddings,0.945554,11,18,1,11.00,1.00,1.0,100.000000,5.555556,100.000000,2.672449,1.039203,2.898515,-0.521182,1
78,XGB,agg,0.949260,87,34,314,62.00,21.00,74.0,71.264368,61.764706,23.566879,16.880789,3.077383,87.720347,0.874730,1
74,LLM,agg,0.956438,87,34,314,86.00,33.50,307.5,98.850575,98.529412,97.929936,1.254855,0.000000,92.739767,1.000000,1


In [90]:
result.to_csv('results_frame.csv')

In [ ]:
result